In [1]:
import sys
sys.path.append('../src')

import numpy as np
import pandas as pd
import data
import utils
from main import extract_summary, report_rouge_scores

%load_ext autoreload
%autoreload 2

/usr/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Get list of titles, reference summaries, and body text
source_titles, source_refs, source_text = data.get_cnn_data(50)
print(len(source_text))

50


In [95]:
from gensim.summarization.textcleaner import clean_text_by_sentences
from gensim.summarization.summarizer import _build_corpus, _build_hasheable_corpus, _set_graph_edge_weights
from gensim.summarization.commons import build_graph, remove_unreachable_nodes
from gensim.summarization.keywords import get_graph
from gensim.summarization.pagerank_weighted import pagerank_weighted, build_adjacency_matrix
from gensim.summarization.bm25 import get_bm25_weights


# hashable_corpus = _build_hasheable_corpus(_build_corpus(clean_text_by_sentences(source_text[0])))

# graph = build_graph(hashable_corpus)
# _set_graph_edge_weights(graph)
#remove_unreachable_nodes(graph)
#pagerank_scores = pagerank_weighted(graph)
#pagerank_scores
# coeff_adjacency_matrix = build_adjacency_matrix(graph)
# matrix = coeff_adjacency_matrix.toarray()
# damping=0.85
# probabilities = (1 - damping) / float(len(graph))
# matrix += probabilities
#hashable_corpus.sort(key=lambda doc: pagerank_scores.get(doc, 0), reverse=True)
#graph.nodes()
#hashable_corpus

# graph = get_graph("The road to hell is paved with good intentions.")
# result = pagerank_weighted(graph)
# result
#corpus = utils.split_sentence(source_text[0])
corpus = [sen.token for sen in clean_text_by_sentences(source_text[0])]
X = get_bm25_weights([s.split() for s in corpus])
len(X)

19

In [98]:
corpus = [sen.token for sen in clean_text_by_sentences(source_text[0])]
source = [sen.text for sen in clean_text_by_sentences(source_text[0])]

In [101]:
from scipy.linalg import eig
X = np.array(X)
val, vec = eig(X)
idx = np.argsort(-vec[:, 0])
[print(source[i] + '\n') for i in idx]

if you're the venezuelan government, you take over a toilet paper factory.

when you're running low on toilet paper and getting desperate, what do you do?

the aim, he explained, is to review the "production, marketing and distribution (of) toilet paper.".

by the "people's defense," arreaza was referring to a government agency created on september 13 by president nicolas maduro to "defeat the economic war that has been declared in the country," according to a report from state-run atv.

at that time, fleming announced the country would import 50 million rolls of toilet paper to meet demand.

on saturday, vice president jorge arreaza announced the "temporary occupation" of the paper manufacturing company's plant in the state of aragua.

the bathroom essential is one of the basic goods and foodstuffs that have been disappearing from store shelves since earlier this year.

other hygiene products, such as toothpaste and soap, may be similarly brought in bulk for the same reason, the minis

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Summarization

In [69]:
doc_idx = 0
doc = source_text[doc_idx]
ref = source_refs[doc_idx]
title = source_titles[doc_idx] if source_titles else None

In [104]:
methods = ['first-k', 'SMRS', 'TextRank', 'FWSR-BM25', 'FWSR-SIF']
extract_summary(doc, ref, title, report_rouge=True, print_rouge=True, methods=methods, print_summary=True);

Soruce Text: 19 sentences, 171 distinct vocab
# of selected sentences: 4
=============== Referecne Text ==============
venezuela occupies paper manufacturing company's plant in aragua, vice president says. vp: nation won't "allow hoarding or failures in the production and distribution" of essentials. government accuses companies of hoarding, blames media for fanning fears. private firms says ill-conceived price control, currency policies have stifled production.
-----
Word count:46

========== Extracted summary: First k ==========
When you're running low on toilet paper and getting desperate, what do you do?
If you're the venezuelan government, you take over a toilet paper factory.
On saturday, vice president jorge arreaza announced the "temporary occupation" of the paper manufacturing company's plant in the state of aragua.
The aim, he explained, is to review the "production, marketing and distribution (of) toilet paper.".
-----
Word count:62
First_k selection computation time: 0.000


### ROUGE Score

In [23]:
%%time
# k=5
# ratio=0.3
methods = ['first-k', 'SMRS', 'TextRank', 'FWSR-BM25', 'FWSR-SIF']
extract_summary(doc, ref, title, report_rouge=True, rouge_embed=False, 
                methods=methods, print_summary=False, print_rouge=True);


=============== ROUGE Scores ===============

first-k
Overlap 1-gram 			F1: 0.300
Overlap 1-gram 			Precision: 0.255
Overlap 1-gram 			Recall: 0.364
Overlap bi-gram 		F1: 0.065
Overlap bi-gram 		Precision: 0.054
Overlap bi-gram 		Recall: 0.081
Longest Common Subsequence 	F1: 0.283
Longest Common Subsequence 	Precision: 0.255
Longest Common Subsequence 	Recall: 0.364

SMRS
Overlap 1-gram 			F1: 0.122
Overlap 1-gram 			Precision: 0.102
Overlap 1-gram 			Recall: 0.152
Overlap bi-gram 		F1: 0.000
Overlap bi-gram 		Precision: 0.000
Overlap bi-gram 		Recall: 0.000
Longest Common Subsequence 	F1: 0.114
Longest Common Subsequence 	Precision: 0.102
Longest Common Subsequence 	Recall: 0.152

TextRank
Overlap 1-gram 			F1: 0.232
Overlap 1-gram 			Precision: 0.165
Overlap 1-gram 			Recall: 0.394
Overlap bi-gram 		F1: 0.014
Overlap bi-gram 		Precision: 0.010
Overlap bi-gram 		Recall: 0.027
Longest Common Subsequence 	F1: 0.180
Longest Common Subsequence 	Precision: 0.165
Longest Common Subsequence

### Word Embedding ROUGE Score

In [17]:
%%time
methods = ['first-k', 'SMRS', 'TextRank', 'FWSR-BM25', 'FWSR-SIF']
_ = extract_summary(doc, ref, title, report_rouge=True, rouge_embed=True, 
                    methods=methods, print_summary=False, print_rouge=True);


=============== ROUGE Scores ===============

first-k
Overlap 1-gram 			F1: 0.529
Overlap 1-gram 			Precision: 0.454
Overlap 1-gram 			Recall: 0.634
Overlap bi-gram 		F1: 0.627
Overlap bi-gram 		Precision: 0.541
Overlap bi-gram 		Recall: 0.744
Longest Common Subsequence 	F1: 0.289
Longest Common Subsequence 	Precision: 0.264
Longest Common Subsequence 	Recall: 0.634

SMRS
Overlap 1-gram 			F1: 0.333
Overlap 1-gram 			Precision: 0.311
Overlap 1-gram 			Recall: 0.358
Overlap bi-gram 		F1: 0.497
Overlap bi-gram 		Precision: 0.489
Overlap bi-gram 		Recall: 0.505
Longest Common Subsequence 	F1: 0.274
Longest Common Subsequence 	Precision: 0.247
Longest Common Subsequence 	Recall: 0.358

TextRank
Overlap 1-gram 			F1: 0.541
Overlap 1-gram 			Precision: 0.471
Overlap 1-gram 			Recall: 0.634
Overlap bi-gram 		F1: 0.635
Overlap bi-gram 		Precision: 0.546
Overlap bi-gram 		Recall: 0.759
Longest Common Subsequence 	F1: 0.254
Longest Common Subsequence 	Precision: 0.235
Longest Common Subsequence

## ROUGE Score Across Documents

In [3]:
#k = 5
start = 0
num_articles = 50
#articles = source_text[start : start + num_articles]
#references = source_refs[start : start + num_articles]
#titles = source_titles[start : start + num_articles] if source_titles else None

In [4]:
%%time
methods = ['first-k', 'SMRS', 'TextRank', 'FWSR-BM25', 'FWSR-SIF']
rouge_mean, rouge_median, rouge_std = report_rouge_scores(source_text[start : start + num_articles], 
                                                          source_refs[start : start + num_articles], 
                                                          methods=methods)

index =  ['1-gram F1', '1-gram Precision', '1-gram Recall', 'bi-gram F1', 'bi-gram Precision', 'bi-gram Recall', 
          'longest common F1', 'longest common Precision', 'longest common Recall', 'runtime', 'word count']

print('=' * 22 + ' Mean ' + '=' * 22)
rouge_mean.index = index
display(rouge_mean)

print('=' * 21 + ' Median ' + '=' * 21)
rouge_median.index = index
display(rouge_median)

print('=' * 15 + ' Standard Deviation ' + '=' * 15)
rouge_std.index = index
display(rouge_std)

====================== Mean ======================


,first-k,SMRS,TextRank,FWSR-BM25,FWSR-SIF
1-gram F1,0.294351,0.112886,0.274215,0.243111,0.252973
1-gram Precision,0.233306,0.112132,0.209661,0.176354,0.215249
1-gram Recall,0.426295,0.129385,0.414050,0.407620,0.324946
bi-gram F1,0.102360,0.011433,0.082003,0.065067,0.077831
bi-gram Precision,0.079474,0.010627,0.060036,0.046395,0.065130
bi-gram Recall,0.155870,0.013811,0.136162,0.114456,0.102587
longest common F1,0.234303,0.095571,0.207752,0.175147,0.211727
longest common Precision,0.215461,0.105194,0.188953,0.159853,0.198332
longest common Recall,0.395592,0.118992,0.373967,0.369935,0.298449
runtime,0.000002,0.416465,0.008374,0.000676,0.000767


===================== Median =====================


,first-k,SMRS,TextRank,FWSR-BM25,FWSR-SIF
1-gram F1,0.285714,0.121324,0.271007,0.255435,0.260870
1-gram Precision,0.233093,0.111111,0.203774,0.178453,0.218803
1-gram Recall,0.410020,0.123724,0.418011,0.397368,0.345788
bi-gram F1,0.079078,0.000000,0.059289,0.057983,0.057983
bi-gram Precision,0.064583,0.000000,0.046032,0.040294,0.054960
bi-gram Recall,0.118118,0.000000,0.103191,0.100847,0.067033
longest common F1,0.217275,0.093993,0.199213,0.173060,0.204430
longest common Precision,0.196154,0.100000,0.182145,0.155870,0.185528
longest common Recall,0.376689,0.105296,0.356231,0.354839,0.309402
runtime,0.000002,0.409360,0.006971,0.000533,0.000674


=============== Standard Deviation ===============


,first-k,SMRS,TextRank,FWSR-BM25,FWSR-SIF
1-gram F1,9.179386e-02,0.061925,0.075324,0.073689,0.092870
1-gram Precision,8.034488e-02,0.056874,0.062575,0.057610,0.079440
1-gram Recall,1.456564e-01,0.090397,0.121346,0.125786,0.133374
bi-gram F1,8.803017e-02,0.026728,0.068386,0.051503,0.071069
bi-gram Precision,6.902955e-02,0.026159,0.050387,0.037592,0.057738
bi-gram Recall,1.372448e-01,0.031687,0.115417,0.087262,0.101268
longest common F1,8.795233e-02,0.052386,0.069108,0.058382,0.082450
longest common Precision,8.087494e-02,0.053474,0.061470,0.052118,0.075375
longest common Recall,1.458678e-01,0.079670,0.122362,0.114406,0.125185
runtime,5.313258e-07,0.044452,0.004674,0.000748,0.000293


CPU times: user 5min 54s, sys: 27 s, total: 6min 21s
Wall time: 8min 55s


In [5]:
rouge_mean.to_csv('cnn_lexical_rouge_mean_50.csv', index=False)
rouge_median.to_csv('cnn_lexical_rouge_median_50.csv', index=False)
rouge_std.to_csv('cnn_lexical_rouge_std_50.csv', index=False)

### Word Embedding ROUGE Score Across Documents

In [ ]:
%%time
methods = ['first-k', 'SMRS', 'TextRank', 'FWSR-BM25', 'FWSR-SIF']
rouge_mean_embed, rouge_median_embed, rouge_std_embed = report_rouge_scores(source_text[start : start + num_articles], 
                                                                            source_refs[start : start + num_articles],  
                                                                            rouge_embed=True, methods=methods)

index =  ['1-gram F1', '1-gram Precision', '1-gram Recall', 'bi-gram F1', 'bi-gram Precision', 'bi-gram Recall', 
          'longest common F1', 'longest common Precision', 'longest common Recall', 'runtime', 'word count']

print('=' * 22 + ' Mean ' + '=' * 22)
rouge_mean_embed.index = index
display(rouge_mean_embed)

print('=' * 21 + ' Median ' + '=' * 21)
rouge_median_embed.index = index
display(rouge_median_embed)

print('=' * 15 + ' Standard Deviation ' + '=' * 15)
rouge_std_embed.index = index
display(rouge_std_embed)

In [17]:
rouge_mean.to_csv('cnn_semantic_rouge_mean_50.csv', index=False)
rouge_median.to_csv('cnn_semantic_rouge_median_50.csv', index=False)
rouge_std.to_csv('cnn_semantic_rouge_std_50.csv', index=False)

## Data

In [27]:
references[3]

'chicago wins bid for new george lucas museum. mayor rahm emanuel calls it "a milestone" for his city. lucas says a prime location in chicago was a deciding factor. museum will house lucas\' art collection and movie memorabilia.'